# Get average PM2.5 by satellite pass over window

**Terra flyover:** approx. 17-19Z -> 18Z and 19Z in DEQ discrete data

**Aqua flyover:** approx. 19-21Z -> 20Z and 21Z in DEQ discrete data

'date' column is in UTC and 'hour' column is in MDT/MST. 'Datetime' is unreliable.

In [ ]:
import pandas as pd

In [ ]:
# open csv in dataframe, only keep useful columns
data = pd.read_csv('Montana_Air_Quality_Monitoring_Data.csv',
                   usecols=['date', 'rawvalue', 'sitename', 'longitude', 'latitude'])

data.head(3)

,date,latitude,rawvalue,sitename,longitude
0,2012/01/01 18:00:00+00,46.002602,12.224,Butte,-112.501247
1,2012/01/01 18:00:00+00,46.850500,2.636,NCore,-111.987164
2,2012/01/01 18:00:00+00,47.175632,17.330,Seeley Lake,-113.476161


In [ ]:
# ignore null values and negatives
data = data.loc[data['rawvalue'] >= 0]

In [ ]:
# separate date column into date and time
def getTime(datetime):
    dt = str(datetime)
    time = int(dt.split(' ')[1].split(':')[0])
    return time

def getDate(datetime):
    dt = str(datetime)
    date = dt.split(' ')[0].replace('/', '-')
    return date

data['time'] = data.date.apply(getTime)
data['date'] = data.date.apply(getDate)

In [ ]:
# get unique dates
unique_dates = data.date.unique()

# get unique ground stations
unique_sitenames = data.sitename.unique()

# get latitude and longitude by station in dictionary
lats = {}
longs = {}
for site in unique_sitenames:
    lat = data.loc[data.sitename == site, 'latitude'].iloc[0]
    long = data.loc[data.sitename == site, 'longitude'].iloc[0]
    lats[site] = lat
    longs[site] = long

In [ ]:
# empty lists for holding averaged data
datetimes = []
sitenames = []
rawvalues = []

# filter by date and station
for date in unique_dates:
    open_sites = data.loc[data.date == date, 'sitename'].unique()
    for site in open_sites:
        subtable = data.loc[(data.date == date) & (data.sitename == site)]

        # average time window 1
        window1 = subtable.loc[(subtable.time == 18) | (subtable.time == 19)]
        rawvalues.append(window1.rawvalue.mean())
        sitenames.append(site)
        datetimes.append(date + 'T18:00')

        # average time window 2
        window2 = subtable.loc[(subtable.time == 20) | (subtable.time == 21)]
        rawvalues.append(window2.rawvalue.mean())
        sitenames.append(site)
        datetimes.append(date + 'T20:00')

# put data back into dataframe
output_data = pd.DataFrame({'datetime': datetimes,
                              'station': sitenames,
                              'pm25': rawvalues})

In [ ]:
# add back coordinates
output_data['latitude'] = output_data.station.map(lats)
output_data['longitude'] = output_data.station.map(longs)

In [ ]:
# some rawvalues are created due to observations being present at a station
# during only one of the time window
output_data = output_data.loc[output_data.pm25.isna()==False]

In [ ]:
# fix index column
output_data.reset_index(drop=True, inplace=True)

In [ ]:
# split in two (easier for processing in GEE later)
one = output_data.iloc[(output_data.index < (len(output_data) // 2)), :]
two = output_data.iloc[(output_data.index >= (len(output_data) // 2)), :]
two.reset_index(drop=True, inplace=True)
two.index.name = 'index'
one.index.name = 'index'

In [ ]:
# output as a csvs
one.to_csv('PM_ave_1.csv')
two.to_csv('PM_ave_2.csv')